Let's load our data from postgres.

In [2]:
import pandas as pd
import psycopg2
from ast import literal_eval
import numpy as np

cred = literal_eval(open('dbcred.py', 'r').read())

In [3]:
conn = psycopg2.connect(dbname = 'cava', user = cred['uid'], password = cred['pwd'], host = 'localhost', port = '5432') 

query = "select * from transactions"

df = pd.read_sql_query(query, conn)

Here is where I will make my first assumption about the data. I believe that the transid field is actually a customer id. 

Given that the prompt instructs us to see if a person is more likely to purchase the same item twice, it is unlikely
that the transid column is actually a transaction id - in other words - it would be near impossible to complete this task if 
transid was a simple one-time purchase with no way of tying it back to a particular customer.

By making this assumption, we now have customer level information with potentially multiple purchases available for each customer. We now have at least two rows of information about each customer, one row for the 'Base' and another for the accompanying protein that they purchased. If a customer id has more than one 'Base' item type, it represents another visit/separate purchase by the customer.

Let's convert transid to a more readable customer-id. Also, we will assign the categorical 'Item' field to an unique numerical value. This will help our processing later on.

In [4]:
df['customer_id'] = pd.factorize(df['transid'])[0]

In [5]:
df['item_int'] = pd.factorize(df['item'])[0]

Now let's convert each value in the Item field to an appropriate category. By looking at Cava's menu online, I was able to sort
each value in to 3 categories: 'Base', 'Protein', or 'Sides'. I made the assumption that lamb sliders was a protein option, 
as it is available at Cava Grill but not Cava Mezze. We will be using a handy pandas feature that allows for quick comparisons
and assignment using our dataframe's index.

In [6]:
df['Type'] = None

In [7]:
base_idx = df[df['item'].isin(['Bowl', 'Pita', 'Salad', 'Greens&Grains', 'Minis'])].index.tolist()
protein_idx = df[df['item'].isin(['falafel', 'braised_lamb', 'Chicken', 'braised_beef', 'meatballs', 'lamb_sliders'])].index.tolist()
sides_idx = df[df['item'].isin(['soup', 'cookie', 'large_drink', 'small_drink', 'chips'])].index.tolist()

In [8]:
df.loc[base_idx, 'Type'] = 'Base'
df.loc[protein_idx, 'Type'] = 'Protein'
df.loc[sides_idx, 'Type'] = 'Side'

In [9]:
df['Meal_id'] = None

In [10]:
base_id = np.arange(856700)

In [11]:
df.set_value(base_idx,'Meal_id', base_id)

,item,transid,id,customer_id,item_int,Type,Meal_id
0,Bowl,14549363-7413-45df-906c-cdc0e896aeb1,1,0,0,Base,0
1,falafel,14549363-7413-45df-906c-cdc0e896aeb1,2,0,1,Protein,None
2,Pita,80dd482d-b377-4aab-ae78-3cd1b5415615,3,1,2,Base,1
3,Chicken,80dd482d-b377-4aab-ae78-3cd1b5415615,4,1,3,Protein,None
4,Salad,80dd482d-b377-4aab-ae78-3cd1b5415615,5,1,4,Base,2
5,meatballs,80dd482d-b377-4aab-ae78-3cd1b5415615,6,1,5,Protein,None
6,Bowl,adff98c1-5a02-4d7b-9076-d67a5db41f19,7,2,0,Base,3
7,Chicken,adff98c1-5a02-4d7b-9076-d67a5db41f19,8,2,3,Protein,None
8,Bowl,3083e488-37b8-400a-86ad-c5e073fccea5,9,3,0,Base,4
9,meatballs,3083e488-37b8-400a-86ad-c5e073fccea5,10,3,5,Protein,None


In [12]:
df.loc[protein_idx, 'Meal_id'] = df['Meal_id'].shift(1)

In [13]:
df.loc[sides_idx, 'Meal_id'] = df['Meal_id'].shift(1)
df.loc[sides_idx, 'Meal_id'] = df['Meal_id'].shift(1)

In [14]:
df.isnull().values.any()

False

In [15]:
mealcombolist = df.groupby('Meal_id')['item_int'].apply(list)

In [16]:
df_combo = pd.DataFrame(mealcombolist)

df_combo.reset_index(level=0, inplace=True)

df_combo

,Meal_id,item_int
0,0,"[0, 1]"
1,1,"[2, 3]"
2,2,"[4, 5]"
3,3,"[0, 3]"
4,4,"[0, 5]"
5,5,"[0, 5]"
6,6,"[0, 5]"
7,7,"[6, 3]"
8,8,"[0, 5]"
9,9,"[2, 7, 8]"


In [17]:
df2 = pd.merge(df, df_combo, on ='Meal_id', how = 'outer')

In [18]:
df2['meal_combo'] = df2['item_int_y']
del df2['item_int_y']


,item,transid,id,customer_id,item_int_x,Type,Meal_id,meal_combo
0,Bowl,14549363-7413-45df-906c-cdc0e896aeb1,1,0,0,Base,0,"[0, 1]"
1,falafel,14549363-7413-45df-906c-cdc0e896aeb1,2,0,1,Protein,0,"[0, 1]"
2,Pita,80dd482d-b377-4aab-ae78-3cd1b5415615,3,1,2,Base,1,"[2, 3]"
3,Chicken,80dd482d-b377-4aab-ae78-3cd1b5415615,4,1,3,Protein,1,"[2, 3]"
4,Salad,80dd482d-b377-4aab-ae78-3cd1b5415615,5,1,4,Base,2,"[4, 5]"


In [25]:
df2['meal_combo_str'] = df2['meal_combo'].astype(str)

In [26]:
df2[:20]

,item,transid,id,customer_id,item_int_x,Type,Meal_id,meal_combo,meal_combo_str
0,Bowl,14549363-7413-45df-906c-cdc0e896aeb1,1,0,0,Base,0,"[0, 1]","[0, 1]"
1,falafel,14549363-7413-45df-906c-cdc0e896aeb1,2,0,1,Protein,0,"[0, 1]","[0, 1]"
2,Pita,80dd482d-b377-4aab-ae78-3cd1b5415615,3,1,2,Base,1,"[2, 3]","[2, 3]"
3,Chicken,80dd482d-b377-4aab-ae78-3cd1b5415615,4,1,3,Protein,1,"[2, 3]","[2, 3]"
4,Salad,80dd482d-b377-4aab-ae78-3cd1b5415615,5,1,4,Base,2,"[4, 5]","[4, 5]"
5,meatballs,80dd482d-b377-4aab-ae78-3cd1b5415615,6,1,5,Protein,2,"[4, 5]","[4, 5]"
6,Bowl,adff98c1-5a02-4d7b-9076-d67a5db41f19,7,2,0,Base,3,"[0, 3]","[0, 3]"
7,Chicken,adff98c1-5a02-4d7b-9076-d67a5db41f19,8,2,3,Protein,3,"[0, 3]","[0, 3]"
8,Bowl,3083e488-37b8-400a-86ad-c5e073fccea5,9,3,0,Base,4,"[0, 5]","[0, 5]"
9,meatballs,3083e488-37b8-400a-86ad-c5e073fccea5,10,3,5,Protein,4,"[0, 5]","[0, 5]"


In [28]:
samemeal = df2.groupby(['customer_id'])['meal_combo_str'].apply(lambda x: len(x.unique()))

In [45]:
df_samemeal = pd.DataFrame(samemeal)

There are 659,687 unique customers in this dataset. Now that we have finished massaging the data in to something usable, let's see how many of these customers have ordered the same set of items (base/protein/side) more than once.

In [48]:
len(df['customer_id'].unique())

659687

About 18% of customers have ordered the same combination of items more than once, with the count at 120,036.

In [38]:
df_samemeal[df_samemeal['meal_combo_str']>1].count()

meal_combo_str    120036
dtype: int64

The figure drops sharply down to 20,938 for customers ordering the same combination of items more than twice, about 3%.

In [39]:
df_samemeal[df_samemeal['meal_combo_str']>2].count()

meal_combo_str    20938
dtype: int64